# Samalas 1258 ENSO Analysis

This notebook consolidates helper functions and analysis workflows for investigating how the 1258 Samalas eruption impacts ENSO and surface temperatures across different eruption seasons.


In [ ]:

from datetime import datetime
from itertools import product
import xarray as xr
import numpy as np
import pandas as pd
from scipy import stats
import cftime
from cftime import DatetimeNoLeap
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.colors as mcolors
from collections import OrderedDict, namedtuple
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from shapely.geometry import LinearRing, Polygon
from netCDF4 import Dataset as nc
from netCDF4 import num2date

# General utilities

def get_current_datetime():
    now = datetime.now()
    return now.strftime('%D'), now.strftime('%H:%M:%S')

# File paths and naming conventions
FILEPATH = '/glade/derecho/scratch/calipfleger/ilme/'
BASEDIR = '/glade/campaign/cesm/collections/cesmLME/CESM-CAM5-LME/atm/proc/tseries/monthly/'
BASEDIR_OCN = '/glade/derecho/scratch/samantha/iLME_seasvolc/CESM-CAM5-LME/atm/proc/tseries/monthly/'
FILE_PREFIX = '.cam.h0.'
I_MODEL = '/b.ie12.B1850C5CN.f19_g16.LME.'
I_MODEL1 = '/b.ie12.B1850CN.f19_g16.'
MODEL = '/b.e11.BLMTRC5CN.f19_g16.'

VARPLOT = '
 1258 Samalas Eruption_1x 10 Members'
VARIABLE = 'Surface Temperature Relative to 30 Year Climatology'
UNIT = 'C'
PLT_YLABEL = 'Surface Temperature (C)'
VARI = 'TS'
MONTHS = ['-5','-4','-3','-2', '-1','0', '1', '2', '3', '4', '5']
COLORS = ['blue', 'black', 'red' ,'green']
COLORS_RIBBON = ['lightblue' ,'lightgrey','mistyrose', 'lightgreen']
VAR_NAME = ['January_1x',  'April_1x', 'July_1x', 'October_1x']
VAR_NAMES = ['DJF', 'MAM', 'JJA', 'SON']

# Region definitions
REGBOX = [-5, 5, 190, 240]
WLATS = [-5, -5, 5, 5]
WLONS = [-240, -190, -190, -240]
WRING = LinearRing(list(zip(WLONS, WLATS)))

# CESM time helpers
dates_sam = [DatetimeNoLeap(year, month, 1) for year, month in product(range(1250, 1265), range(1, 13))]
da_sam = xr.DataArray(np.arange(180), coords=[dates_sam], dims=['time'], name='time')

years_cntl = np.linspace(0,348, 360)
dates_cntl = [DatetimeNoLeap(years_cntl, month, 1) for years_cntl, month in product(range(1228, 1258), range(1, 13))]
da_cntl = xr.DataArray(np.arange(360), coords=[dates_cntl], dims=['time'], name='time')

# NetCDF utility

def save_ensemble(x, var_name, filename):
    xrda = xr.concat(x, dim='member')
    xrda.to_netcdf(f"{FILEPATH}{var_name}{filename}.nc")
    return xrda


In [ ]:

import numpy as np
import xarray as xr

def _area_weighted_mean(data, lat_name='lat', lon_name='lon'):
    weights = np.cos(np.deg2rad(data[lat_name]))
    return data.weighted(weights).mean(dim=[lat_name, lon_name])

def nino_region(data, lat_bounds, lon_bounds, lat_name='lat', lon_name='lon'):
    subset = data.sel({lat_name: slice(*lat_bounds), lon_name: slice(*lon_bounds)})
    return _area_weighted_mean(subset, lat_name=lat_name, lon_name=lon_name)

def nino34(data, lat_name='lat', lon_name='lon'):
    return nino_region(data, (-5,5), (190,240), lat_name, lon_name)

def nino3(data, lat_name='lat', lon_name='lon'):
    return nino_region(data, (-5,5), (210,270), lat_name, lon_name)

def nino4(data, lat_name='lat', lon_name='lon'):
    return nino_region(data, (-5,5), (160,210), lat_name, lon_name)

def calculate_nino34(data):
    nino34 = data.sel(lat=slice(-5,5), lon=slice(190,240))
    weights = np.cos(np.deg2rad(nino34.lat))
    return nino34.weighted(weights).mean(dim=['lat','lon'])

def season_nino34(m):
    DJF = m.sel(time=m.time.dt.season == 'DJF')
    JJA = m.sel(time=m.time.dt.season == 'JJA')
    return DJF, JJA


In [ ]:

from itertools import combinations
from typing import Iterable, Dict, Tuple
import numpy as np
import xarray as xr
from scipy import stats

from nino_indices import nino34
from samalas_setup import FILEPATH, VAR_NAME

def load_ts_ensemble(onset, members, variable='TS'):
    paths = [f"{FILEPATH}{onset}/{variable}_m{m:02d}.nc" for m in members]
    datasets = [xr.open_dataarray(p) for p in paths]
    return xr.concat(datasets, dim='member')

def compute_ts_anomaly(exp, control):
    control_clim = control.groupby('time.month').mean('time')
    return exp.groupby('time.month') - control_clim

def classify_pre_eruption_phase(sst, months=12, threshold=1.0, eruption_index=None):
    if eruption_index is None:
        eruption_index = months
    n34 = nino34(sst)
    pre = n34.isel(time=slice(eruption_index - months, eruption_index))
    mean = pre.mean('time')
    std = pre.std('time')
    phase = xr.full_like(mean, 'Neutral', dtype=object)
    phase = xr.where(mean > threshold * std, 'El Nino', phase)
    phase = xr.where(mean < -threshold * std, 'La Nina', phase)
    return phase

def composite_post_eruption(ts_anom, phases, months=24, eruption_index=None):
    if eruption_index is None:
        eruption_index = 0
    post = ts_anom.isel(time=slice(eruption_index, eruption_index + months))
    comps = {}
    for phase in ['El Nino', 'Neutral', 'La Nina']:
        mask = phases == phase
        if mask.any():
            comps[phase] = post.sel(member=mask).mean('member')
    return comps

def analyze_eruption_seasonality(months=VAR_NAME, members=range(1,11), variable='TS', pre_months=12, post_months=24, control=None, eruption_index=None):
    if eruption_index is None:
        eruption_index = pre_months
    results = {}
    for onset in months:
        ts = load_ts_ensemble(onset, members, variable=variable)
        if control is not None:
            ts_anom = compute_ts_anomaly(ts, control)
        else:
            base = ts.isel(time=slice(eruption_index - pre_months, eruption_index)).mean('time')
            ts_anom = ts - base
        phase_labels = classify_pre_eruption_phase(ts, months=pre_months, eruption_index=eruption_index)
        composites = composite_post_eruption(ts_anom, phase_labels, months=post_months, eruption_index=eruption_index)
        results[onset] = composites
    return results

def _global_mean(data, lat_name='lat', lon_name='lon'):
    weights = np.cos(np.deg2rad(data[lat_name]))
    return data.weighted(weights).mean(dim=[lat_name, lon_name])

def _nino34_anomaly(ts, control=None, pre_months=12, eruption_index=None):
    if eruption_index is None:
        eruption_index = pre_months
    if control is not None:
        ts_anom = compute_ts_anomaly(ts, control)
    else:
        base = ts.isel(time=slice(eruption_index - pre_months, eruption_index)).mean('time')
        ts_anom = ts - base
    return nino34(ts_anom)

def ttest_onset_differences(onsets=VAR_NAME, members=range(1,11), variable='TS', pre_months=12, post_months=24, control=None, eruption_index=None):
    if eruption_index is None:
        eruption_index = pre_months
    ts_means = {}
    n34_anoms = {}
    for onset in onsets:
        ts = load_ts_ensemble(onset, members, variable=variable)
        if control is not None:
            ts_anom = compute_ts_anomaly(ts, control)
        else:
            base = ts.isel(time=slice(eruption_index - pre_months, eruption_index)).mean('time')
            ts_anom = ts - base
        ts_mean = _global_mean(ts_anom).isel(time=slice(eruption_index, eruption_index + post_months))
        n34 = _nino34_anomaly(ts, control, pre_months, eruption_index).isel(time=slice(eruption_index, eruption_index + post_months))
        ts_means[onset] = ts_mean
        n34_anoms[onset] = n34
    results = {}
    for a, b in combinations(onsets, 2):
        ts_t, ts_p = stats.ttest_ind(ts_means[a], ts_means[b], axis=0, equal_var=False)
        n34_t, n34_p = stats.ttest_ind(n34_anoms[a], n34_anoms[b], axis=0, equal_var=False)
        results[(a, b)] = xr.Dataset({'ts_t': ('time', ts_t), 'ts_p': ('time', ts_p), 'n34_t': ('time', n34_t), 'n34_p': ('time', n34_p)}, coords={'time': ts_means[a].time})
    return results
